In [3]:
import pandas as pd
df = pd.read_csv('statements.csv', encoding='utf8')

In [14]:
statements = df['statements'].values

In [15]:
statements[:5]

array(['Fashion week in NYC is in February ',
       'Roosevelt Field  site did Lindbergh begin his flight from in 1927',
       'Saudi Arabia  country is the holy city of Mecca located in',
       'the sun is  5778.0 hot',
       'banana  flavor filling did the original Twinkies have'],
      dtype=object)

# test spacy Entity Recogmition

In [16]:
import spacy
from nltk import Tree
en_nlp = spacy.load('en_core_web_lg')

In [17]:
def get_spacy_entities(text, count=3):
    nlp_doc = en_nlp(text)
    return[ent.text for ent in nlp_doc.ents[:count]]

get_spacy_entities(statements[1])

['Roosevelt Field', 'Lindbergh', '1927']

# Test Wikification

In [18]:
def get_wiki_entities(text, count=3):
    import tagme
    from operator import itemgetter
    tagme.GCUBE_TOKEN = "c294fe63-88fa-40e1-b1d4-34b70c887b29-843339462"
    newlist = [{'score': item.score,'mention':item.mention,'entity_title':item.entity_title} for item in tagme.annotate(text).get_annotations(0.1)]
    newlist = sorted(newlist, key=itemgetter('score'), reverse=True)
    return [{'mention':item['mention'],'entity_title':item['entity_title']} for item in newlist[:count]]

get_wiki_entities(statements[1])

[{'entity_title': 'Roosevelt Field (airport)', 'mention': 'Roosevelt Field'},
 {'entity_title': 'Charles Lindbergh', 'mention': 'Lindbergh'},
 {'entity_title': 'Flight International', 'mention': 'flight'}]

# semantic role labeling
allennlp # http://demo.allennlp.org/semantic-role-labeling, sorces doesnot work, need to to try docker
nlpnet #only portoguese
senna = python(pntl == practnlptools) cant use becouse of dependensies of senna and standford parser 

# Rake keyword extraction

In [20]:
from rake_nltk import Rake

def get_rake_entities(text, count=3):
    r = Rake() 
    r.extract_keywords_from_text(text)
    return r.get_ranked_phrases()[:count] # To get keyword phrases ranked highest to lowest.
    
get_rake_entities(statements[1])

['roosevelt field site', 'lindbergh begin', 'flight']

# Plan

# Search in Wiipedia

In [21]:
def get_merget_entities(text):
    entities = get_wiki_entities(text)
    if len(entities)<2:
        spacy_entities =  get_spacy_entities(text)
        for spacy_entity in spacy_entities:
            for entity in entities:
                if (spacy_entity not in entity['mention']) and (entity['mention'] not in spacy_entity):
                    entities.append({'mention':spacy_entity,'entity_title':spacy_entity})
    return entities

In [35]:
import wikipedia
wikipedia.set_lang("en")
# get data from wikipedia https://pypi.python.org/pypi/wikipedia
def get_wiki_content(entity, pages_count):
    fined_page_names = wikipedia.search(entity)
    if len(fined_page_names)>0:
        try:
            content=[]
            for page_name in fined_page_names[:pages_count]:
                wiki_page = wikipedia.page(page_name)
                content.append(wiki_page.content)
            return '. '.join(content)
        except:
            return ''
    else:
        return ''
    
get_wiki_content('Roosevelt Field (airport)',2)

'Roosevelt Field is a former airport, located 2.3 miles (3.7 km) east-southeast of Mineola, Long Island, New York. Originally called the Hempstead Plains Aerodrome, or sometimes Hempstead Plains field or the Garden City Aerodrome, it was a training field (Hazelhurst Field) for the Air Service, United States Army during World War I.\nIn 1919, it was renamed in honor of President Theodore Roosevelt\'s son, Quentin, who was killed in air combat during World War I.\nRoosevelt Field was the takeoff point for many historic flights in the early history of aviation, including Charles Lindbergh\'s 1927 solo transatlantic flight. It was also used by other pioneering aviators, including Amelia Earhart and Wiley Post.\n\n\n== History ==\nThe Hempstead Plains Aerodrome originally encompassed 900 to 1,000 acres (405 ha) east of and abutting Clinton Road, south of and adjacent to Old Country Road, and west of Merrick Avenue. A bluff 15 feet in elevation divided the plain into two large fields. The U.

In [36]:
from nltk.tokenize import sent_tokenize
def text_to_sents(text):
    sents = sent_tokenize(text)
    #lematized_sents =[]
    #for sent in sents:
    #    doc = en_nlp(sent)
    #    lematized_sent=' '.join([token.lemma_ for token in doc])
    #    lematized_sents.append(lematized_sent)
    #return lematized_sents
    return sents
text_to_sents("Roosevelt Field is a former airport, located 2.3 miles (3.7 km) east-southeast of Mineola, Long Island, New York. Originally called the Hempstead Plains Aerodrome, or sometimes Hempstead Plains field or the Garden City Aerodrome, it was a training field (Hazelhurst Field) for the Air Service, United States Army during World War I.\nIn 1919, it was renamed in honor of President Theodore Roosevelt\'s son, Quentin, who was killed in air combat during World War I.\nRoosevelt Field was the takeoff point for many historic flights in the early history of aviation, including Charles Lindbergh\'s 1927 solo transatlantic flight. ")

['Roosevelt Field is a former airport, located 2.3 miles (3.7 km) east-southeast of Mineola, Long Island, New York.',
 'Originally called the Hempstead Plains Aerodrome, or sometimes Hempstead Plains field or the Garden City Aerodrome, it was a training field (Hazelhurst Field) for the Air Service, United States Army during World War I.',
 "In 1919, it was renamed in honor of President Theodore Roosevelt's son, Quentin, who was killed in air combat during World War I.\nRoosevelt Field was the takeoff point for many historic flights in the early history of aviation, including Charles Lindbergh's 1927 solo transatlantic flight."]

In [40]:
def get_comention_score_in_sents(entity, sents):
    counter=0
    for sent in  sents:
        if entity in sent:
            counter=counter+1
    return counter

In [39]:
def check_true(text):
    true_counter=0
    entities = get_merget_entities(text)
    for entity in entities:
        wiki_content = get_wiki_content(entity['entity_title'],1)
        wiki_sents = text_to_sents(wiki_content)
        for entity_2 in entities:
            if entity_2['mention']!=entity['mention']:
                true_counter=true_counter+ get_comention_score_in_sents(entity_2['mention'],wiki_sents)
                
    return true_counter

def check_true_by_original_entity(text):
    true_counter=0
    entities = get_merget_entities(text)
    for entity in entities:
        wiki_content = get_wiki_content(entity['mention'],5)
        wiki_sents = text_to_sents(wiki_content)
        for entity_2 in entities:
            if entity_2['mention']!=entity['mention']:
                true_counter=true_counter+ get_comention_score_in_sents(entity_2['mention'],wiki_sents)
                
    return true_counter

In [25]:
check_true(statements[1])

53

In [26]:
from tqdm import tqdm
coefs = []
for i in tqdm(range(df.shape[0])):
    coefs.append(check_true(statements[i]))

  2%|▏         | 21/867 [02:01<1:21:22,  5.77s/it]/Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
100%|██████████| 867/867 [1:20:36<00:00,  5.58s/it]


In [27]:
df['coefs']=coefs

In [ ]:
df.to_csv('data/statements.csv', encoding='utf8', index=False)

# Result by searchink key words in Wiki abstract


In [40]:
df[:10][['statements','coefs']]

,statements,coefs
0,Fashion week in NYC is in February,0
1,Roosevelt Field site did Lindbergh begin his ...,53
2,Saudi Arabia country is the holy city of Mecc...,36
3,the sun is 5778.0 hot,0
4,banana flavor filling did the original Twinki...,14
5,Home on the Range is the state song of Kansas,129
6,Crimean war is connected with the book `` Cha...,10
7,the first atomic bomb detonated in New Mexico,45
8,Europe continent is Scotland in,261
9,Puerto Rico island did the U.S. gain after th...,224


In [38]:
correct_answer_count = df[df['coefs']>0].shape[0]/df.shape[0]
correct_answer_count

0.8708189158016147

# Check reason why other statements was not found

In [42]:
step2_df = df[df['coefs']==0]

In [43]:
step2_df[['statements','coefs']]

,statements,coefs
0,Fashion week in NYC is in February,0
3,the sun is 5778.0 hot,0
11,Pyrotechnists make fireworks called,0
15,it get in Death Valley does 56.6°C hot,0
24,JFK born in 1917,0
35,76 s inch 80 s inch are the measurem...,0
61,a hand is 4 in high,0
70,"LuSBE, is the abbreviation for the London stoc...",0
75,1981 year was the IBM PC invented,0
76,do you have to be to join the U.S. Navy 17 old,0


Entities searched in sentenses:

In [45]:
get_merget_entities('JFK born in 1917')

[{'entity_title': 'JFK (film)', 'mention': 'JFK'},
 {'entity_title': '1917', 'mention': '1917'}]

In [46]:
#increase retrived pages count in Wikipedia

In [47]:
check_true_by_original_entity('JFK born in 1917')

2

In [55]:
from tqdm import tqdm
coefs = []
for i in tqdm(range(step2_df.shape[0])):
    coefs.append(check_true_by_original_entity(step2_df['statements'].values[i]))
step2_df['coef_2']=coefs

  4%|▍         | 5/112 [00:36<13:07,  7.36s/it]Z:\anaconda\envs\py_36\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file Z:\anaconda\envs\py_36\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
100%|██████████| 112/112 [18:46<00:00, 10.06s/it]
Z:\anaconda\envs\py_36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

In [58]:
step2_df[['statements','coefs','coef_2']][step2_df['coef_2']>0]

,statements,coefs,coef_2
0,Fashion week in NYC is in February,0,2
24,JFK born in 1917,0,2
119,Tong Tong is the name of Ling Ling 's mate,0,2
142,Prairie View A & M University is in Texas,0,21
190,United Kingdom country celebrates Guy Fawkes Day,0,3
192,the Exxon Valdez Oil spill in 1989,0,6
206,kiev is in UA,0,1
211,42 natural number immediately follows 41 and ...,0,34
272,Edwin is Buzz Aldrin 's real first name,0,7
290,jelly bean is Ronald Reagan 's favorite candy,0,2


# Lets try to increase this value by using sparql and query DBPedia

In [60]:
step3_df = step2_df[step2_df['coef_2']==0]
step3_df

,id,type,question,answer,answer_value,statements,coefs,coef_2
3,1992,factoid,How hot is the sun?,"(5,?778|6,?[05]00)\s*K|(5,?505|6,?[50]00).*C|(...",5778.0,the sun is 5778.0 hot,0,0
11,1663,factoid,What are the people who make fireworks called?,Pyrotechnists,Pyrotechnists,Pyrotechnists make fireworks called,0,0
15,2364,factoid,How hot does it get in Death Valley?,\b134\b|56\.7|\b116\b|\b47\b|56\.6\°C,56.6°C,it get in Death Valley does 56.6°C hot,0,0
35,2186,factoid,What are the measurements for a king-size bed?,"76\s*(inch|"").*80\s*(inch|"")",76 s inch 80 s inch,76 s inch 80 s inch are the measurem...,0,0
61,1582,factoid,How high is a hand?,4\s*in(ches)?,4 in,a hand is 4 in high,0,0
70,1667,factoid,What is the abbreviation for the London stock ...,L.?S.?E.?,"LuSBE,","LuSBE, is the abbreviation for the London stoc...",0,0
75,1549,factoid,What year was the IBM PC invented?,1981,1981,1981 year was the IBM PC invented,0,0
76,2333,factoid,How old do you have to be to join the U.S. Navy?,\b17\b,17,do you have to be to join the U.S. Navy 17 old,0,0
80,10064,factoid,how much is 1+1?,\b2\b|two,2,1+1 is 2 much,0,0
89,2360,factoid,What is the gift for the 20th anniversary?,china|platinum,platinum,platinum is the gift for the 20th anniversary,0,0


In [62]:
import spacy
from nltk import Tree
en_nlp = spacy.load('en_core_web_lg')

In [297]:
def get_root(doc):
    return [token for token in doc if token.dep_=='ROOT']
    
get_root(en_nlp('the sun is 5778.0 hot'))

[is]

In [343]:
def print_doc(doc):
    for token in doc:
        rights=[]
        for i in token.rights:
            rights.append(i)
        print('{7}<-{0}({1})->{8}    dep:{2}, pos:{3}, ent:{4}, orth:{5}, tag_:{6}, rights:{7}'.format(
            token.text,token.head, token.dep_,token.pos_,token.ent_type_, token.orth_, token.tag_,token.left_edge,token.right_edge,rights))

In [ ]:
#spacy preprocesing block

#Merge compaunds together
#def merge_doc(text):

In [311]:
import re
def cleanup_text(text):
    return re.sub(' +',' ',text.strip(' \t\n\r'))

cleanup_text('JFK born in 1917 ')

'JFK born in 1917'

In [415]:
# function helps merge compaund words toggether and manage them as a single item

def merge_nouns(doc):
    viewed_tokens_for_merge=[]
    merge_ranges=[]
    for token in doc:
        if token in viewed_tokens_for_merge:
            continue
        tmp_compaund_token=token
        
        while(tmp_compaund_token.dep_ in ['compound','nummod']):
            viewed_tokens_for_merge.append(token)
            tmp_compaund_token = tmp_compaund_token.head
            
        if token!=tmp_compaund_token:
            merge_ranges.append((token.idx, tmp_compaund_token.idx+tmp_compaund_token.__len__()))
            
    for merge_range in merge_ranges:
        doc.merge(merge_range[0],merge_range[1])
    return doc

def merge_werbs(doc):
    viewed_tokens_for_merge=[]
    merge_ranges=[]
    for token in doc:
        if token in viewed_tokens_for_merge:
            continue
        tmp_compaund_token=token
        
        while (tmp_compaund_token.dep_ in ['auxpass']) and ( (tmp_compaund_token.tag_ in ['VBD'])):
            print(tmp_compaund_token)
            viewed_tokens_for_merge.append(tmp_compaund_token)
            viewed_tokens_for_merge.append(tmp_compaund_token.head)
            tmp_compaund_token = tmp_compaund_token.head
            
        if token!=tmp_compaund_token:
            merge_ranges.append((token.idx, tmp_compaund_token.idx+tmp_compaund_token.__len__()))
            
    for merge_range in merge_ranges:
        doc.merge(merge_range[0],merge_range[1])
    return doc

def merge_NN_with_entities(doc):
    viewed_tokens_for_merge=[]
    merge_ranges=[]
    for token in doc:
        if token in viewed_tokens_for_merge:
            continue
        tmp_compaund_token=token
        
        while (tmp_compaund_token.ent_type_ != '') and ( tmp_compaund_token.dep_ in ['amod']) and ( tmp_compaund_token.head.tag_ in ['NN']) :
            viewed_tokens_for_merge.append(tmp_compaund_token)
            viewed_tokens_for_merge.append(tmp_compaund_token.head)
            tmp_compaund_token = tmp_compaund_token.head
            
        if token!=tmp_compaund_token:
            merge_ranges.append((token.idx, tmp_compaund_token.idx+tmp_compaund_token.__len__()))
            
    for merge_range in merge_ranges:
        doc.merge(merge_range[0],merge_range[1])
    return doc


def get_prprocessed_spacy_doc(text):
    cleaned_text = cleanup_text(text)
    doc = en_nlp(cleaned_text)
    doc = merge_nouns(doc)
    doc = merge_NN_with_entities(doc)
#doc = merge_werbs(doc)
    return doc
    
print_doc(get_prprocessed_spacy_doc('the U.N. created in 1945'))

the<-the(U.N.)->the    dep:det, pos:DET, ent:, orth:the, tag_:DT, rights:the
the<-U.N.(created)->U.N.    dep:nsubj, pos:PROPN, ent:ORG, orth:U.N., tag_:NNP, rights:the
the<-created(created)->1945    dep:ROOT, pos:VERB, ent:, orth:created, tag_:VBN, rights:the
in<-in(created)->1945    dep:prep, pos:ADP, ent:, orth:in, tag_:IN, rights:in
1945<-1945(in)->1945    dep:pobj, pos:NUM, ent:DATE, orth:1945, tag_:CD, rights:1945


In [386]:
def tree_move(token, next_token, handler):
    # endless loop fix fumction
    #print(token, next_token)
    if (token == next_token):
        return None
    return handler(next_token)

In [371]:
def get_identifier(token):
    #print(token,token.tag_,token.right_edge,token.left_edge)
    if token.tag_ in ['VBZ','VB','VBP','VBD']:
        return  tree_move(token,token.left_edge,get_identifier)
    if token.dep_ in ['nsubj']:
        return token
    if token.tag_ in ['NN','NNS','NNP']:
        return token
    if token.tag_ == 'DT':
        return get_identifier(token.head)
    if token.ent_type_=='QUANTITY':
        return token
    else: 
        return None

In [416]:
def IS_case_rule_recursive(token):
    #print('IS_case_rule_recursive:   ', token,token.tag_,token.right_edge,token.left_edge)
    if token.tag_ in ['VBZ','VB','VBP','VBD','VBN','VBG']:
        return  token
    elif (token.tag_ in ['JJ']) and (token.ent_type_==''):
        return  token
    else:
        return tree_move(token,token.right_edge,IS_case_rule_recursive)
        

def get_attribute_IS_case(token):
    #print('get_attribute_IS_case')
    attribute =  tree_move(token,token.right_edge,IS_case_rule_recursive)
    return attribute if attribute != None else token

def get_attribute(token):
    #print(token,token.tag_,token.right_edge,token.left_edge)
    if token.tag_ in ['VBZ','VB','VBP','VBD','VBN','VBG']:
        if token.lower_ in ['is']:
            return get_attribute_IS_case(token)
        return  token
    else:
        return None

In [397]:
def get_value_as_an_attribute_of_root(token):
    for child in token.children:
        if (child.dep_=='attr') and (child.tag_ in ['CD']) and (child.ent_type_!=''):
            return child
    return None
    
def get_value_on_right(token):
    #print(token, token.tag_, token.ent_type_)
    if (token.tag_ in ['NN','NNS', 'CD']) or (token.ent_type_!=''):
        return token
    if token.tag_ in ['JJ','VBN']:
        return tree_move(token,token.left_edge,get_value)
    return None

def get_value(token):
    value = get_value_as_an_attribute_of_root(token)
    if value != None:
        return value
    return get_value_on_right(token.right_edge)

In [417]:
def get_triplets_from_text(text):
    #NN VB value
    #NN (is)attribute_name value
    
    doc = get_prprocessed_spacy_doc(text)
    roots = get_root(doc)
    if len(roots)!=1:
        return (None,None, None)
    else:
        root = roots[0]
    identifier = get_identifier(root.left_edge)
    attribute= get_attribute(root)
    value = get_value(root)
    return identifier, attribute, value

get_triplets_from_text('the U.N. created in 1945')

(U.N., created, 1945)

In [402]:
step2_df['statements'].values[13]

'chuppot is the name of the canopy at a Jewish wedding'

In [419]:
tests=[
    ('the sun is 5778.0 hot',('sun', 'hot', '5778.0')),
    ('Pyrotechnists make fireworks called',('Pyrotechnists', 'make', 'fireworks')),
    ('Fashion week in NYC is in February ',('Fashion week', 'is', 'February')),
    ('JFK born  in 1917 ',('JFK', 'born', '1917')),
    ('76 s  inch     80 s  inch     are the measurements for a king-size bed',('76 s inch', 'are','king-size bed')),
    ('a hand is  4 in high',('hand', 'high', '4')),
    ('LuSBE, is the abbreviation for the London stock exchange',('LuSBE', 'is', 'London stock exchange')),
    ('platinum is the gift for the 20th anniversary',('platinum', 'is', '20th anniversary'),
    ('the U.N. created in 1945',('U.N.', 'created', '1945'))
)
]
def test_runner(tests):
    error_counter=0
    for test in tests:
        identifier, attribute, value=get_triplets_from_text(test[0])
        identifier = identifier.text if identifier!=None else None
        attribute = attribute.text if identifier!=None else None
        value = value.text if identifier!=None else None
        if (identifier, attribute, value)!=test[1]:
            print(test)
            error_counter+=1
    if error_counter==0:
        print('You are my HERO!!!!')

test_runner(tests)

You are my HERO!!!!


# result of extractiong some triplets

In [452]:
identifiers=[]
attributes=[]
values=[]
for statement in step3_df['statements'].values:
    identifier, attribute, value = get_triplets_from_text(statement)
    identifiers.append(identifier)
    attributes.append(attribute)    
    values.append(value)
    
step3_df['identifiers']=identifiers
step3_df['attributes']=attributes
step3_df['values']=values

Z:\anaconda\envs\py_36\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
Z:\anaconda\envs\py_36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
Z:\anaconda\envs\py_36\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [ ]:
step3_df[['statements','identifiers','attributes','values']]

In [437]:
cant_work_with_that = step3_df[step3_df[['identifiers','attributes','values']].isnull().any(axis=1)]
cant_work_with_that[:5][['statements','identifiers','attributes','values']]

,statements,identifiers,attributes,values
15,it get in Death Valley does 56.6°C hot,it,get,None
75,1981 year was the IBM PC invented,1981 year,invented,None
76,do you have to be to join the U.S. Navy 17 old,None,have,None
96,the score in 9 : 0,score,None,0
120,light travel through space does 671 million m...,None,does,None


In [439]:
cant_work_with_that.shape

(52, 11)

In [440]:
need_to_proces_with_rdf_df=  step3_df[~step3_df[['identifiers','attributes','values']].isnull().any(axis=1)]
need_to_proces_with_rdf_df[:5][['statements','identifiers','attributes','values']]

,statements,identifiers,attributes,values
3,the sun is 5778.0 hot,sun,hot,5778.0
11,Pyrotechnists make fireworks called,Pyrotechnists,make,fireworks
35,76 s inch 80 s inch are the measurem...,76 s inch,are,king-size bed
61,a hand is 4 in high,hand,high,4
70,"LuSBE, is the abbreviation for the London stoc...",LuSBE,is,London stock exchange


In [441]:
need_to_proces_with_rdf_df.shape

(37, 11)

# To hard to cover different cases be writing rules, how to handle it? is it right way of extractiong knowladge?

In [447]:
def check_true_for_triplets_basic_wersion(identifier, attributes,values):    
    wiki_content = get_wiki_content(identifier,5)
    wiki_sents = text_to_sents(wiki_content)        
    return get_comention_score_in_sents(values,wiki_sents)

In [453]:
need_to_proces_with_rdf_df['coef3']=need_to_proces_with_rdf_df.apply(lambda row: check_true_for_triplets_basic_wersion(row['identifiers'].text,row['attributes'].text,row['values'].text), axis=1)

Z:\anaconda\envs\py_36\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file Z:\anaconda\envs\py_36\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
Z:\anaconda\envs\py_36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel

In [454]:
need_to_proces_with_rdf_df[['statements','identifiers','attributes','values','coef3']]

,statements,identifiers,attributes,values,coef3
3,the sun is 5778.0 hot,sun,hot,5778.0,0
11,Pyrotechnists make fireworks called,Pyrotechnists,make,fireworks,0
35,76 s inch 80 s inch are the measurem...,76 s inch,are,king-size bed,0
61,a hand is 4 in high,hand,high,4,0
70,"LuSBE, is the abbreviation for the London stoc...",LuSBE,is,London stock exchange,0
80,1+1 is 2 much,1,much,2,140
89,platinum is the gift for the 20th anniversary,platinum,is,20th anniversary,0
113,chuppot is the name of the canopy at a Jewish ...,chuppot,is,Jewish wedding,4
132,48 is the critical angle of rainbow,48,is,rainbow,0
166,pieces are there in a dozen 12 many,pieces,are,,295


In [ ]:
'Dead Sea is the lowest point on dry land' -> ('Dead Sea','is','land') КЛАС!

In [ ]:
Питання які залишились:
    Як правильно будувати з тексту триплети
    як обєднувати триплети в структури RDF
    як матчити питання-відповідь в RDF просторі

Чи Варто використовувати RDF як Knowladge base, будувати її з 0